# 環境準備

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pwd
%cd /content/drive/My Drive/キーフレーズ抽出
!ls

/content
/content/drive/My Drive/キーフレーズ抽出
 data.json   yahoo_shopping.ipynb	      花安勇人.ipynb
 github     'yahoo_shopping.ipynb のコピー'


In [3]:
!pip install -U ginza

import pkg_resources, imp
imp.reload(pkg_resources)

import spacy
nlp = spacy.load('ja_ginza')

!pip list | grep ginza

     |████████████████████████████████| 10.4MB 5.9MB/s 
     |████████████████████████████████| 51.5MB 86kB/s 
     |████████████████████████████████| 71kB 7.4MB/s 
     |████████████████████████████████| 1.1MB 32.3MB/s 
     |████████████████████████████████| 481kB 46.0MB/s 
  Created wheel for ginza: filename=ginza-4.0.5-cp36-none-any.whl size=15898 sha256=706dc398be7dba71364827c51506c9d04bf031ee603edefdfa1d857ecfe4af4d
  Stored in directory: /root/.cache/pip/wheels/a2/4f/b8/74029780ece4fedfa1f16ba6b9b86fdeee8260ade265e3756e
  Created wheel for ja-ginza: filename=ja_ginza-4.0.0-cp36-none-any.whl size=51530814 sha256=e90d2cb435535546156ba450d4a220cef18001afa17464b00d26625dc34fb273
  Stored in directory: /root/.cache/pip/wheels/28/5a/c0/95ac590b39eff99c77d729f284341d15a6903e4011d70ff421
  Created wheel for SudachiPy: filename=SudachiPy-0.5.1-cp36-cp36m-linux_x86_64.whl size=871045 sha256=f7658e0a019ec859f439f3d4291663a7560f80dff609e80d03d1365429a64442
  Stored in directory: /root/.cach

In [4]:
!pip install git+https://github.com/boudinfl/pke.git

  Cloning https://github.com/boudinfl/pke.git to /tmp/pip-req-build-1iqdt4sq
  Running command git clone -q https://github.com/boudinfl/pke.git /tmp/pip-req-build-1iqdt4sq
     |████████████████████████████████| 245kB 4.2MB/s 
  Created wheel for pke: filename=pke-1.8.1-cp36-none-any.whl size=8763600 sha256=09d0f140efc98d1157510e96620ea2dbc71e3314ffa95e3c86d1cf99d79f2c79
  Stored in directory: /tmp/pip-ephem-wheel-cache-afvjfen0/wheels/8d/24/54/6582e854e9e32dd6c632af6762b3a5d2f6b181c2992e165462
Successfully built pke


In [5]:
# pkeのストップワードの定義
!python -m nltk.downloader stopwords

import nltk
import pke

pke.base.lang_stopwords['ja_ginza'] = 'japanese'
stopwords = list(list(spacy.lang.ja.stop_words.STOP_WORDS))
nltk.corpus.stopwords.words_org = nltk.corpus.stopwords.words
nltk.corpus.stopwords.words = lambda lang : stopwords if lang == 'japanese' else nltk.corpus.stopwords.words_org(lang)

/usr/lib/python3.6/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# レビューの取得

In [6]:
import requests
import json

In [7]:
url = 'https://shopping.yahooapis.jp/ShoppingWebService/V1/json/reviewSearch'
api_id = ''

In [8]:
num_request = 50
json_list = []
start = 1

# スイッチのレビューを取得
while(True):
    response = requests.get(url, params = {'appid':api_id, 'jan':4902370542912, 'results':num_request, 'start':start})
    json_data = response.json()['ResultSet']['Result']
    print(json_data)
    json_list += json_data
    if len(json_data) < num_request:
        break
    start += num_request

[{'ReviewTitle': '定価でPayPayボーナスもたくさん付…', 'Description': '定価で購入しPayPayボーナスもたくさん付きお得に購入出来ました。配送も早く梱包もしっかりしており満足です。SwitchはTVラックの見える所に置いていますがカラーがネオンだと目立ちますね。インテリア的にはグレーが良かったかもしれません。', 'Ratings': {'Rate': '5.00', 'Average': '4.82'}, 'Count': {'All': '698'}, 'Recommend': '0', 'Update': '2021-01-24T19:20:42+09:00', 'Url': 'https://shopping.yahoo.co.jp/review/item/list?store_id=joshin&page_key=4902370542912-54-40519', 'ReviewType': 'buyer', 'Purpose': '', 'SendTo': '', 'Target': {'_attributes': {'type': 'item'}, 'Name': '任天堂 (新モデル)Nintendo Switch 本体(Joy-Con(L) ネオンブルー/ (R) ネオンレッド) 返品種別B', 'Code': 'joshin_4902370542912-54-40519', 'Url': 'https://store.shopping.yahoo.co.jp/joshin/4902370542912-54-40519.html', 'Image': {'Id': 'joshin_4902370542912-54-40519', 'Small': {'Url': 'https://item-shopping.c.yimg.jp/i/c/joshin_4902370542912-54-40519', 'Width': '76', 'Height': '76'}, 'Medium': {'Url': 'https://item-shopping.c.yimg.jp/i/g/joshin_4902370542912-54-40519', 'Width': '146', 'Height': '146'}}, 'St

# データの成型

In [9]:
import pandas as pd

In [10]:
image_url = json_list[0]['Target']['Image']['Medium']['Url']
product_name = json_list[0]['Target']['Name']
data_list = []

# 必要なデータだけをリストに格納
for data in json_list:
    data_list.append([data['ReviewTitle'], data['Description'], data['Ratings']['Rate'], data['Recommend'], data['Update']])

# データフレームに変換
df = pd.DataFrame(data_list, columns=['ReviewTitle', 'Description', 'Rate', 'Recommend', 'Update'])
df.head()

,ReviewTitle,Description,Rate,Recommend,Update
0,定価でPayPayボーナスもたくさん付…,定価で購入しPayPayボーナスもたくさん付きお得に購入出来ました。配送も早く梱包もしっかり...,5.00,0,2021-01-24T19:20:42+09:00
1,購入出来て良かったです。今後もソフトを…,購入出来て良かったです。今後もソフトを買いたいです。,5.00,0,2021-01-24T17:36:24+09:00
2,良いです。,思っていた以上に良い物でした。\n息子も気に入って使ってます。,5.00,0,2021-01-23T21:40:34+09:00
3,子どもの誕生日プレゼントで購入しました…,子どもの誕生日プレゼントで購入しました\n梱包も特に問題なく、稼働も問題ありません,5.00,0,2021-01-23T20:10:39+09:00
4,やっと買えました。,やっと買えました。\n孫たちとお正月にゲームで楽しむことができました。,5.00,0,2021-01-23T18:06:55+09:00


# レビューからキーフレーズ抽出

## キーフレーズのスコアの算出

In [11]:
def union_text_score(data, model='MultipartiteRank'):
    score_list = []
    text = ''
    for index, row in df.iterrows():
        text += row['Description'] + ' :\n'

    # モデルの定義
    if model == 'MultipartiteRank': extractor = pke.unsupervised.MultipartiteRank()
    elif model == 'TopicRank': extractor = pke.unsupervised.TopicRank()

    doc_text = nlp(text)
    cat_stopwords = []

    # 入力データの定義
    extractor.load_document(input=text, language='ja_ginza', normalization=None)
    
    # キーフレーズの選択（
    extractor.candidate_selection(pos={'NOUN', 'PROPN', 'NUM'}, stoplist=stopwords+cat_stopwords)

    # スコアの計算
    extractor.candidate_weighting(threshold=0.5, method='average')

    return (text, extractor.get_n_best(n=20))

In [12]:
# キーフレーズを抽出
text, key_score_list = union_text_score(data=df, model='MultipartiteRank')
print(key_score_list)

[('ゲーム', 0.028996595500865183), ('子ども', 0.024347841858817474), ('家族 みんな', 0.021625735857663177), ('switch', 0.02048872647459492), ('ポイント', 0.01784953124154699), ('クリスマスプレゼント', 0.01747891615706424), ('子供達', 0.013932617648525932), ('午前中', 0.013181336874546996), ('マスク', 0.012973254069651989), ('買い物', 0.0124622253763917), ('paypay 支払い', 0.011275491199303569), ('ショップ', 0.011110988842945918), ('みんな', 0.01052116528962267), ('メール', 0.010101121010365555), ('プレゼント', 0.009875499677063729), ('火曜日', 0.009822931821487985), ('日曜日', 0.009785407419463207), ('人気 商品', 0.009428757155553613), ('クリスマス', 0.009422031555469427), ('家族 全員', 0.009419767704428093)]


In [13]:
import pprint
from collections import OrderedDict

key_data_dict = OrderedDict()
for ks in key_score_list:
    key_data_dict[ks[0].replace(' ', '')] = {'score':ks[1], 'sum':0, 'count':0, 'average':0, 'review':[]}
key_list = [ks[0].replace(' ', '') for ks in key_score_list]

for key in key_list:
    for index, row in df.iterrows():    
        description = row['Description'].replace(' ', '')
        pos = description.find(key)

        # レビューがキーフレーズを含んでいた場合、キーフレーズの周辺の文を抽出
        if pos > -1:
            len_key = len(key)
            len_des = len(description)
            start = 0
            end = 0

            if pos != 0:
                i = 0
                while(True):
                    i += 1
                    if pos-i < 1:
                        start = pos - i
                        break
                    elif row['Description'][pos-i] in ['.', '。', '\n']:
                        start = pos - i + 1
                        break
            i = 0
            while(True):
                i += 1
                if pos+len_key+i >= len_des:
                    end = pos + len_key + i
                    break
                elif row['Description'][pos+len_key+i] in ['.', '。', '\n', '!', '?', '！', '？']:
                    end = pos + len_key + i + 1
                    if row['Description'][end-1] == '\n':
                        end -= 1
                    break
            
            # Webアプリ用にデータを辞書にまとめる
            extraction = row['Description'][start:end]
            key_data_dict[key]['sum'] += float(row['Rate'])
            key_data_dict[key]['count'] += 1
            key_data_dict[key]['review'].append({'title':row['ReviewTitle'],
                                                'update':row['Update'],
                                                'score':row['Rate'],
                                                'recommend':row['Recommend'],
                                                'extraction':extraction,
                                                'description':row['Description']})
    
    # キーフレーズを含んだレビューが５個しかない場合は取り除く
    if key_data_dict[key]['count'] < 5:
        del key_data_dict[key]
        continue
    
    # キーフレーズごとの平均レビュー点数を算出
    key_data_dict[key]['average'] = key_data_dict[key]['sum'] / key_data_dict[key]['count']
    # レビューを参考になった人数順（降順）でソート
    key_data_dict[key]['review'] = sorted(key_data_dict[key]['review'],
                                            key=lambda x:x['recommend'],
                                            reverse=True)

In [14]:
print(len(key_data_dict))
print(key_data_dict.keys())
pprint.pprint(key_data_dict)

8
odict_keys(['ゲーム', '子ども', 'ポイント', 'クリスマスプレゼント', '買い物', 'みんな', 'プレゼント', 'クリスマス'])
OrderedDict([('ゲーム',
              {'average': 4.818181818181818,
               'count': 11,
               'review': [{'description': 'やっと買えました。\n孫たちとお正月にゲームで楽しむことができました。',
                           'extraction': '孫たちとお正月にゲームで楽しむことができました。',
                           'recommend': '0',
                           'score': '5.00',
                           'title': 'やっと買えました。',
                           'update': '2021-01-23T18:06:55+09:00'},
                          {'description': '10年ぶりくらいにゲーム機を買いました。\n'
                                          'リングフィットアドベンチャーでダイエットと思って購入しましたが、思ったより負荷がかかるので、少し痩せることができました。',
                           'extraction': '10年ぶりくらいにゲーム機を買いました。',
                           'recommend': '0',
                           'score': '5.00',
                           'title': '10年ぶりくらいにゲーム機を買いました…',
                           'update': '2021-01-20T20:07:48+09:00'},
              

In [15]:
# データをjsonファイルに
with open('data.json', 'w') as f:
    json.dump(key_data_dict, f, indent=4)